In [1]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [2]:
TOKENIZER = AutoTokenizer.from_pretrained("model_final")

You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers


In [3]:

MODEL = AutoModelForSeq2SeqLM.from_pretrained("model_final")

In [4]:
reference = [
    "Hi. For further information consult a urologist online -->",
    "Hi. I read your query with keen interest and understand your concerns. From the available description, I do not think you have crossed the limit of sanity or you developed schizophrenia. Schizophrenia is altogether very different disorder where people have abnormal perceptions such as hearing voices when no one is present and seeing objects when nothing is there. I do not see any of them with you. In my view, you need to put your worries at rest and do what your mind and heart considers right. It may be abnormal for your mother, but there is no pathology with this. If you feel more assistance is required or my answer to your query is far from setting down the worries, then it is better to see a psychiatrist in person. For further information consult a psychiatrist online -->",
    "Hello. An anal leak is mostly due to underlying reasons, which need to be thoroughly investigated. Although we can together try reaching out on differential diagnosis. Regards. Revert with more information to a cosmetologist online -->"
]

question = [
    "Hello doctor, I am a 22 year old male. I have problem of tight penis skin. Whenever I pull my penis skin back during erection it will be very hard to do. Then, my penis skin gets stuck behind the penis forehead and it is very painful to retract back. I generally use oil for this problem. Please suggest me what to do.",
    "Hi doctor, I am a 24 year old female. I am a healthy and happy person. I eat well and exercise well. I have a lot of friends, cool hobbies, etc. But, I have a weird habit. Whenever I am alone I talk to myself standing in front of the mirror. I imagine a lot. I think so many things like I am in an interview and I try to venture out the best answers and framing smart answers. When I am practicing on my guitar, I imagine lots of people are watching me and appreciating me for playing so well. I imagine futuristic conversations with people where I ask a question and answer them. When I am alone, I keep imagining some stuff. Also, I pick up some topic and start talking to myself in front of the mirror. I personally feel this habit has improved my communication skills, confidence and vocabulary. I am not shy and I do not have stage fear. When I meet my friends or when I am giving presentations at college, I am the one who used to stay very confident. However, some days back, when I was talking to myself in front of the mirror, my mom questioned me. Until now I did not pay much attention to what I was doing. But, ever since my mom saw me doing that I have become conscious. I have stopped talking to myself for two to three days. But, I do not feel like I have been missing out on anything. I am still friendly and confident. But, I am just wondering if I have crossed the line of sanity. I searched about this on the internet and came across something called as schizophrenia. Is this disorder setting with my habit? Do I have schizophrenia or any other disorder? To be honest, after reading about this schizophrenia I am very scared. It sounds like a very serious disorder to me. Please help.",
    "Hello doctor, I have a wet feeling inside the anus for the past one year. I did a colonoscopy and sigmoidoscopy, and the doctor said that everything is fine. They said probably the wet feeling is due to sweat. Swab culture and sensitivity report showed up Klebsiella pneumoniae. I took Augmentin 625 and applied ointments. But I feel it is some liquid discharge from the anus skin. Can you suggest some medication?"
]

In [5]:
# Skor ROUGE untuk evaluasi
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
smoothing = SmoothingFunction().method4 
for question_text, ref in zip(question, reference):
    # Tokenisasi input
    inputs = TOKENIZER(question_text, return_tensors="pt")
    
    # Generate output dari model
    outputs = MODEL.generate(inputs["input_ids"], max_length=512, num_beams=5, early_stopping=True)
    
    # Decode output menjadi teks
    output_text = TOKENIZER.decode(outputs[0], skip_special_tokens=True)
    
    print("Input:", question_text)
    print("Generated Output:", output_text)

    # Menghitung skor ROUGE antara referensi dan output model
    rouge_scores = scorer.score(ref, output_text)
    print(f"ROUGE scores for question: {question_text}")
    print(rouge_scores)
    

    reference_tokens = ref.split()
    output_tokens = output_text.split()
    bleu_score = sentence_bleu([reference_tokens], output_tokens, smoothing_function=smoothing)
    print(f"BLEU score: {bleu_score}\n")

Input: Hello doctor, I am a 22 year old male. I have problem of tight penis skin. Whenever I pull my penis skin back during erection it will be very hard to do. Then, my penis skin gets stuck behind the penis forehead and it is very painful to retract back. I generally use oil for this problem. Please suggest me what to do.
Generated Output: Hello. For further information consult a sexologist online -->
ROUGE scores for question: Hello doctor, I am a 22 year old male. I have problem of tight penis skin. Whenever I pull my penis skin back during erection it will be very hard to do. Then, my penis skin gets stuck behind the penis forehead and it is very painful to retract back. I generally use oil for this problem. Please suggest me what to do.
{'rouge1': Score(precision=0.75, recall=0.75, fmeasure=0.75), 'rouge2': Score(precision=0.5714285714285714, recall=0.5714285714285714, fmeasure=0.5714285714285714), 'rougeL': Score(precision=0.75, recall=0.75, fmeasure=0.75)}
BLEU score: 0.5133450

In [6]:
# Fungsi untuk menjawab pertanyaan
def get_model_response(question):
    # Tokenisasi input
    inputs = TOKENIZER(question, return_tensors="pt")
    
    # Generate output dari model
    outputs = MODEL.generate(inputs["input_ids"], max_length=512, num_beams=5, early_stopping=True)
    
    # Decode output menjadi teks
    output_text = TOKENIZER.decode(outputs[0], skip_special_tokens=True)
    return output_text


In [7]:
# Inpoet user
user_question = input("Masukkan pertanyaan Anda: ")
if user_question.lower() == "exit":
    print("Terima kasih telah menggunakan layanan ini!")
else:
    response = get_model_response(user_question)
    print("\nJawaban dari model:")
    print(response)
    print("\n")

KeyboardInterrupt: Interrupted by user

# What are the symptoms of a common cold?
# How can I treat a headache?
# What should I do if I experience chest pain during physical activity?
# How can I manage my blood pressure if it is consistently high?
# What are the potential causes of sudden weight loss without any changes in diet or exercise?
